In [ ]:
#추진방법
##이미지, 영상에서 사람얼굴 식별(Opencv, Haar-Cascade-Classifiers, 딥러닝 등)
##블러처리 하여 export
##향후 streamlit, bootstrap 등으로 웹으로 시각화하여 구축
##최종적으로 이미지, 영상 업로드하여 가명처리된 이미지, 영상 export 설계
##영상 처리시 속도가 너무 느림, 경량화하는 방법 필요

#참고문헌
## https://velog.io/@smile_b/Haar-Cascade-Classifiers%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%96%BC%EA%B5%B4-%EB%B0%8F-%EB%88%88-%EA%B0%90%EC%A7%80

In [4]:
#오픈 CV 설치
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   - -------------------------------------- 1.8/38.8 MB 11.2 MB/s eta 0:00:04
   ----- ---------------------------------- 5.0/38.8 MB 11.6 MB/s eta 0:00:03
   ------- -------------------------------- 7.3/38.8 MB 11.9 MB/s eta 0:00:03
   ---------- ----------------------------- 10.0/38.8 MB 11.7 MB/s eta 0:00:03
   ------------ --------------------------- 12.6/38.8 MB 11.8 MB/s eta 0:00:03
   --------------- ------------------------ 14.9/38.8 MB 11.9 MB/s eta 0:00:03
   ------------------ --------------------- 17.6/38.8 MB 11.8 MB/s eta 0:00:02
   -------------------- ------------------- 20.2/38.8 MB 11.8 MB/s eta 0:00:02
   ----------------------- ---------------- 22.5/38.8 MB 11.8 MB/s eta 0:00:02
   ------------------------- -------------- 25.2/38.8 MB 11.8 MB/s eta 0:00:02
   ---------------------------- ----------- 27.8/38.8 MB 11.8 MB/s eta 0:00:01
   ------------------------------- -------- 30.4/38.8 MB 11.8 MB

In [2]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
#1-3 가장 최근에 나온 YOLOv8-Face로 활용
!pip install ultralytics
!mkdir -p /content/weights
!wget -O /content/weights/yolov8n.pt https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 37.6 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
from ultralytics import YOLO
import time

# 파일 경로 설정
input_video_path = '/content/sample_data/sample.mp4'
output_video_path = '/content/sample_data/blurred_output.mp4'

# YOLOv8-Face 모델 로드 (GPU 사용 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/content/weights/yolov8n.pt").to(device)

# 동영상 파일 열기
cap = cv2.VideoCapture(input_video_path)

# 동영상의 속성 가져오기
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# 시간 측정 시작
start_time = time.time()
frame_count = 0
frame_skip = 5  # 프레임 샘플링 간격 (5 프레임마다 얼굴 재탐지)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 샘플링: frame_skip 간격으로 얼굴 탐지 수행
    if frame_count % frame_skip == 0:
        # YOLOv8-Face로 얼굴 탐지 (GPU에서 실행)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # RGB 변환 (필요 시)
        results = model(frame_rgb)

        # 얼굴 영역을 저장
        faces = []
        for result in results:
            boxes = result.boxes.xyxy  # 얼굴 바운딩 박스 좌표 (x1, y1, x2, y2)
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)  # 정수형으로 변환
                faces.append((x1, y1, x2, y2))  # 얼굴 위치 저장

    # 이전에 저장된 얼굴 위치에 블러 처리
    for (x1, y1, x2, y2) in faces:
        face = frame[y1:y2, x1:x2]
        blurred_face = cv2.GaussianBlur(face, (25, 25), 50)  # 강한 블러 적용
        frame[y1:y2, x1:x2] = blurred_face

    # 결과 프레임을 출력 파일에 저장
    out.write(frame)
    frame_count += 1

    # 남은 시간 예측
    if frame_count % 10 == 0:  # 매 10 프레임마다 남은 시간 업데이트
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / frame_count) * total_frames
        remaining_time = estimated_total_time - elapsed_time
        print(f"진행률: {frame_count / total_frames * 100:.2f}% | 남은 시간: {remaining_time:.2f}초")

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()

# 총 처리 시간
end_time = time.time()
total_elapsed_time = end_time - start_time

print("가명처리된 동영상이 저장되었습니다:", output_video_path)
print(f"총 처리 시간: {total_elapsed_time:.2f}초")


0: 384x640 (no detections), 127.9ms
Speed: 12.7ms preprocess, 127.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.9ms
Speed: 3.7ms preprocess, 15.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
진행률: 0.83% | 남은 시간: 124.10초

0: 384x640 (no detections), 14.5ms
Speed: 3.8ms preprocess, 14.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.6ms
Speed: 3.4ms preprocess, 18.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
진행률: 1.67% | 남은 시간: 90.47초

0: 384x640 (no detections), 12.6ms
Speed: 3.5ms preprocess, 12.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.6ms
Speed: 2.3ms preprocess, 10.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
진행률: 2.50% | 남은 시간: 72.21초

0: 384x640 (no detections), 10.4ms
Speed: 2.2ms preprocess, 10.4ms inference, 0.6ms postprocess per image at shape (

In [ ]:
#2. 이미지 가명처리

# 파일 경로 설정
input_image_path = '/content/sample_data/sample.jpg'
output_image_path = '/content/sample_data/blurred_output.jpg'

# YOLOv8 모델 로드 (GPU 사용 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/content/weights/yolov8n.pt").to(device)

# 이미지 파일 열기
image = cv2.imread(input_image_path)
if image is None:
    print("이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    # 시간 측정 시작
    start_time = time.time()

    # YOLOv8으로 얼굴 탐지
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # YOLOv8은 RGB 포맷을 사용하므로 변환 필요
    results = model(image_rgb)

    # 얼굴 영역을 블러 처리
    for result in results:
        boxes = result.boxes.xyxy  # 얼굴 바운딩 박스 좌표 (x1, y1, x2, y2)
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)  # 정수형으로 변환
            face = image[y1:y2, x1:x2]
            blurred_face = cv2.GaussianBlur(face, (25, 25), 50)  # 강한 블러 적용
            image[y1:y2, x1:x2] = blurred_face

    # 처리 시간 측정 종료
    end_time = time.time()
    total_elapsed_time = end_time - start_time

    # 결과 이미지 저장
    cv2.imwrite(output_image_path, image)
    print("가명처리된 이미지가 저장되었습니다:", output_image_path)
    print(f"총 처리 시간: {total_elapsed_time:.2f}초")


0: 352x640 2 persons, 1 tie, 82.0ms
Speed: 2.2ms preprocess, 82.0ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)
가명처리된 이미지가 저장되었습니다: /content/sample_data/blurred_output.jpg
총 처리 시간: 0.21초
